In [ ]:
import os
from os.path import dirname
root_path = dirname(dirname(os.getcwd()))
print(root_path)
import sys
sys.path.append(root_path + '/RemainingCycleTimePrediction/2_Scripts/')
import pandas as pd
import numpy as np
import copy
import time
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render

from Event_log_processing_utils import Extract_trace_and_temporal_features, Extract_prefix
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

data_dir = root_path + '/RemainingCycleTimePrediction/1_Data/'
project_dir = root_path + '/RemainingCycleTimePrediction/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## 1. Load data

In [ ]:
data_name = 'BPIC20'
# data_name = 'Helpdesk'
# data_name = 'EMS3141BE'

In [ ]:
tab_all = pd.read_csv(data_dir+data_name+"_processed_all.csv")
tab_train= pd.read_csv(data_dir+data_name+"_processed_train.csv")
tab_valid= pd.read_csv(data_dir+data_name+"_processed_valid.csv")
tab_test = pd.read_csv(data_dir+data_name+"_processed_test.csv")

In [ ]:
# Statistics of the dataset
lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(tab_all)
print("num_cases: {}".format(len(tab_all['Case_ID'].unique())))
print("num_activities: {}".format(len(tab_all["Activity"].unique())))
print("num_events: {}".format(len(tab_all)))
avglen = round(np.mean([len(x) for x in lines]), 2)
print("avg_case_len: {}".format(avglen))
maxlen = max([len(x) for x in lines]) #find maximum line size
print("max_case_len: {}".format(maxlen))
print("avg_case_duration: {}".format(round(np.mean([sublist[-1] for sublist in lines_t2])/86400, 2)))
print("max_case_duration: {}".format(round(max([sublist[-1] for sublist in lines_t2])/86400, 2)))
print("min_case_duration: {}".format(round(min([sublist[-1] for sublist in lines_t2])/86400, 2)))
list_unique_line = []
for line in lines:
    if line not in list_unique_line:
        list_unique_line.append(line)
print("variants: {}".format(len(list_unique_line)))

## 2. Prepare inputs and outputs for model training

In [ ]:
def Prepare_X_Y_remaining_time(tab, list_activities, divisor, divisor2, divisor_rt, encoder, maxlen):
    lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(tab)
    prefixes, outputs = Extract_prefix(lines, lines_t, lines_t2, lines_t3, lines_t4)
    num_samples = len(prefixes[0])
#     [sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4], [next_ope, next_ope_t, end_ope_t]
    print('Vectorization...')
    num_features = len(list_activities)+5 #1 order feature + 4 temporal features
    print('num features: {}'.format(num_features))
    X = np.zeros((num_samples, maxlen, num_features), dtype=np.float32)
    Y = np.zeros(num_samples, dtype=np.float32)
    for i, sentence in enumerate(prefixes[0]):
        leftpad = maxlen-len(sentence)
        end_t = outputs[2][i]
        sentence_t = prefixes[1][i]
        sentence_t2 = prefixes[2][i]
        sentence_t3 = prefixes[3][i]
        sentence_t4 = prefixes[4][i]
        one_hot_act_matrix = encoder.transform(np.array(sentence).reshape((len(sentence), 1))).toarray()
        for t, char in enumerate(sentence):                
            X[i, t+leftpad, :len(list_activities)] = one_hot_act_matrix[t, :]
            X[i, t+leftpad, len(list_activities)] = t+1 # order of the activity in the sequence {1,...,maxlen}
            X[i, t+leftpad, len(list_activities)+1] = sentence_t[t]/divisor
            X[i, t+leftpad, len(list_activities)+2] = sentence_t2[t]/divisor2
            X[i, t+leftpad, len(list_activities)+3] = sentence_t3[t]/86400
            X[i, t+leftpad, len(list_activities)+4] = sentence_t4[t]/7
        Y[i] = end_t/divisor_rt
    return X, Y

In [ ]:
list_activities = list(tab_all["Activity"].unique())
num_features = len(list_activities)+5
#creating instance of one-hot-encoder and fit on the whole dataset
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(np.array(list_activities).reshape((len(list_activities), 1)))

lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(tab_all)
maxlen = max([len(x) for x in lines]) #find maximum line size
lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(tab_train)
divisor = np.mean([item for sublist in lines_t for item in sublist]) #average time between events
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in lines_t2 for item in sublist]) #average time between current and first events
print('divisor2: {}'.format(divisor2))
prefixes, outputs = Extract_prefix(lines, lines_t, lines_t2, lines_t3, lines_t4)
divisor_rt = np.mean(outputs[2])
print('divisor_rt: {}'.format(divisor_rt))
#Train data
X_train, Y_train = Prepare_X_Y_remaining_time(tab_train, list_activities, divisor, divisor2, divisor_rt, encoder, maxlen)
#Valid data
X_valid, Y_valid = Prepare_X_Y_remaining_time(tab_valid, list_activities, divisor, divisor2, divisor_rt, encoder, maxlen)
#Test data
X_test, Y_test = Prepare_X_Y_remaining_time(tab_test, list_activities, divisor, divisor2, divisor_rt, encoder, maxlen)

In [ ]:
class EventLogData(Dataset):
    def __init__ (self, input_x, output):
        self.X = input_x
        self.y = output
        self.y = self.y.to(torch.float32)
        self.y = self.y.reshape((len(self.y),1))

    #get the number of rows in the dataset
    def __len__(self):
        return len(self.X)

    #get a row at a particular index in the dataset
    def __getitem__ (self,idx):
        return [self.X[idx],self.y[idx]]

In [ ]:
valid_loader = DataLoader(EventLogData(torch.tensor(X_valid), torch.tensor(Y_valid)),
                                batch_size=X_valid.shape[0],
                                shuffle=False)
test_loader = DataLoader(EventLogData(torch.tensor(X_test), torch.tensor(Y_test)),
                                batch_size=1,
                                shuffle=False)

## 3. Hyperparameter tuning with Ax package

In [ ]:
# Creating the LSTM class
class LSTM_direct(nn.Module):
    #  Determine what layers and their order in CNN object 
    def __init__(self, parameterization):
        super(LSTM_direct, self).__init__()
        self.hidden_dim = parameterization.get("neurons", 40)
        self.num_layers = parameterization.get("layers", 1)
        self.droppout_prob = parameterization.get("dropout", 0.2)
                
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=self.hidden_dim, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.droppout_prob)        
        self.fc = nn.Linear(self.hidden_dim, 1)

    
    # Progresses data across layers    
    def forward(self, x):
        batch_size = x.size(0) 
        init_states, init_cells = self.init_hidden(batch_size)
        init_states = init_states.to(x.device)
        init_cells = init_cells.to(x.device)
        lstm_output, (last_Hidden_State, last_Cell_State) = self.lstm(x, (init_states, init_cells)) 
        out = self.fc(last_Hidden_State[-1])
        return out

    def init_hidden(self, batch_size):
        init_states = []
        init_cells = []
        for i in range(self.num_layers):
            init_states.append(torch.zeros(batch_size, self.hidden_dim))
            init_cells.append(torch.zeros(batch_size, self.hidden_dim))
        return torch.stack(init_states, dim=0), torch.stack(init_cells, dim=0)      #(num_layers, B, H)
    
def net_train(net, train_loader, valid_loader, parameters, dtype, device, early_stop_patience):
    net.to(dtype=dtype, device=device)
    min_delta = 0
    # Define loss and optimizer
    criterion = nn.L1Loss()
    optimizer = optim.Adam(net.parameters(), lr=parameters.get("lr", 0.001)) # 0.001 is used if no lr is specified    
    num_epochs = 100 # Play around with epoch number
    
    # Train Network
    not_improved_count = 0
    start_time = time.time()
    for epoch in range(num_epochs):
        net.train()
        training_loss = 0
        num_train = 0
        for inputs, labels in train_loader:
            # move data to proper dtype and device
            inputs = inputs.to(dtype=dtype, device=device)
            labels = labels.to(device=device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            output = net(inputs)
            loss = criterion(output, labels)
            # back prop
            loss.backward()
            # optimize
            optimizer.step()
            training_loss+= loss.item()
            num_train+=1
        with torch.no_grad():
            net.eval()
            num_valid = 0
            validation_loss = 0
            for i,(inputs,targets) in enumerate(valid_loader):
                inputs,targets = inputs.to(device),targets.to(device)
                yhat_valid = net(inputs)
                loss_valid = criterion(yhat_valid,targets)
                validation_loss+= loss_valid.item()
                num_valid+= 1
        avg_training_loss = training_loss/num_train
        avg_validation_loss = validation_loss/num_valid        
        print("Epoch: {}, Training MAE : {}, Validation loss : {}".format(epoch,avg_training_loss,avg_validation_loss))
        if (epoch==0): 
            best_loss = avg_validation_loss
            best_model = copy.deepcopy(net)
        else:
            if (best_loss - avg_validation_loss >= min_delta):
                best_model = copy.deepcopy(net)
                best_loss = avg_validation_loss
                not_improved_count = 0
            else:
                not_improved_count += 1
        # Early stopping
        if not_improved_count == early_stop_patience:
            print("Validation performance didn\'t improve for {} epochs. "
                            "Training stops.".format(early_stop_patience))
            break
    training_time = time.time() - start_time
    print("Training time:", training_time)
    return best_model


def lstm_direct_evaluate(net, data_loader, dtype, device):
    criterion = nn.L1Loss()
    net.eval()
    loss = 0
    total = 0
    with torch.no_grad():
        for i,(inputs,targets) in enumerate(data_loader):
            # move data to proper dtype and device
            inputs = inputs.to(dtype=dtype, device=device)
            targets = targets.to(device=device)
            outputs = net(inputs)
            loss += criterion(outputs,targets)
            total += 1
    return loss.item() / total


def train_evaluate(parameterization):

    # constructing a new training data loader allows us to tune the batch size
    train_loader = DataLoader(EventLogData(torch.tensor(X_train), torch.tensor(Y_train)),
                                batch_size=parameterization.get("batchsize", 32),
                                shuffle=True)
    
    # Get neural net
    untrained_net = LSTM_direct(parameterization)
    # train
    trained_net = net_train(net=untrained_net, train_loader=train_loader, valid_loader = valid_loader, 
                            parameters=parameterization, dtype=dtype, device=device, early_stop_patience = 10)
    
    # return the accuracy of the model as it was trained in this run
    return lstm_direct_evaluate(
        net=trained_net,
        data_loader=valid_loader,
        dtype=dtype,
        device=device,
    )

In [ ]:
dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "neurons", "type": "choice", "values": [40, 60, 80, 100], "value_type": "int"},
        {"name": "layers", "type": "choice", "values": [2, 3, 4, 5], "value_type": "int"},
        {"name": "lr", "type": "range", "bounds": [1e-4, 0.1], "value_type": "float", "log_scale": True},
        {"name": "batchsize", "type": "choice", "values": [16, 32, 64], "value_type": "int"}, 
        {"name": "dropout", "type": "range", "bounds": [0, 0.5], "value_type": "float"}
    ],
  
    evaluation_function=train_evaluate,
    objective_name='MAE loss',
    minimize = True,
    random_seed = 123,
    total_trials = 100
)

print(best_parameters)
means, covariances = values
print(means)

In [ ]:
best_objectives = np.array([[trial.objective_mean for trial in experiment.trials.values()]])

best_objective_plot = optimization_trace_single_method(
    y=np.minimum.accumulate(best_objectives, axis=1),
    title="Model performance vs. # of iterations",
    ylabel="MAE loss",
)
render(best_objective_plot)

render(plot_contour(model=model, param_x='dropout', param_y='lr', metric_name='MAE loss'))

In [ ]:
data = experiment.fetch_data()
df = data.df
best_arm_name = df.arm_name[df['mean'] == df['mean'].min()].values[0]
best_arm = experiment.arms_by_name[best_arm_name]
best_arm

## 4. Re-Train model with tuned hyperparameters

In [ ]:
# Creating the LSTM class
class LSTM_direct_model(nn.Module):
    #  Determine what layers and their order in CNN object 
    def __init__(self, hidden_dim, num_layers, droppout_prob):
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        super(LSTM_direct_model, self).__init__()                
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=hidden_dim, 
                            num_layers=num_layers, batch_first=True, dropout=droppout_prob)        
        self.fc = nn.Linear(hidden_dim, 1)

    
    # Progresses data across layers    
    def forward(self, x):
        batch_size = x.size(0) 
        init_states, init_cells = self.init_hidden(batch_size)
        init_states = init_states.to(x.device)
        init_cells = init_cells.to(x.device)
        lstm_output, (last_Hidden_State, last_Cell_State) = self.lstm(x, (init_states, init_cells)) 
        out = self.fc(last_Hidden_State[-1])a
        return out

    def init_hidden(self, batch_size):
        init_states = []
        init_cells = []
        for i in range(self.num_layers):
            init_states.append(torch.zeros(batch_size, self.hidden_dim))
            init_cells.append(torch.zeros(batch_size, self.hidden_dim))
        return torch.stack(init_states, dim=0), torch.stack(init_cells, dim=0)      #(num_layers, B, H)

In [ ]:
batch_size = best_arm.parameters['batchsize']

train_loader = DataLoader(EventLogData(torch.tensor(X_train), torch.tensor(Y_train)),
                                batch_size=batch_size,
                                shuffle=True)

In [ ]:
save_folder = project_dir + '5_Output_files/Remaining_time_prediction/'+data_name+'_Tax_LSTM_direct'
hidden_dim = best_arm.parameters['neurons']
num_layers = best_arm.parameters['layers']
droppout_prob = best_arm.parameters['dropout']
lr_value = best_arm.parameters['lr']
min_delta = 0
num_epochs = 100
early_stop_patience = 10
num_runs = 5
# Define loss and optimizer   
for run in range(num_runs):
    print("Run: {}".format(run+1))
    model = LSTM_direct_model(hidden_dim, num_layers, droppout_prob)
    model.to(dtype=dtype, device=device) 
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=lr_value)
    epochs_plt = []
    mae_plt = []
    valid_loss_plt = []
    not_improved_count = 0
    # Train Network   
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        training_loss = 0
        num_train = 0
        for inputs, labels in train_loader:
            # move data to proper dtype and device
            inputs = inputs.to(dtype=dtype, device=device)
            labels = labels.to(device=device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            output = model(inputs)
            loss = criterion(output, labels)
            # back prop
            loss.backward()
            # optimize
            optimizer.step()
            training_loss+= loss.item()
            num_train+=1
        with torch.no_grad():
            model.eval()
            num_valid = 0
            validation_loss = 0
            for i,(inputs,targets) in enumerate(valid_loader):
                inputs,targets = inputs.to(device),targets.to(device)
                yhat_valid = model(inputs)
                loss_valid = criterion(yhat_valid,targets)
                validation_loss+= loss_valid.item()
                num_valid+= 1
        avg_training_loss = training_loss/num_train
        avg_validation_loss = validation_loss/num_valid        
        print("Epoch: {}, Training MAE : {}, Validation loss : {}".format(epoch,avg_training_loss,avg_validation_loss))
        epochs_plt.append(epoch+1)
        mae_plt.append(avg_training_loss)
        valid_loss_plt.append(avg_validation_loss)
        if (epoch==0): 
            best_loss = avg_validation_loss
            torch.save(model.state_dict(),'{}/best_model_run_{}.pt'.format(save_folder,run+1))
            best_model = copy.deepcopy(model)
        else:
            if (best_loss - avg_validation_loss >= min_delta):
                torch.save(model.state_dict(),'{}/best_model_run_{}.pt'.format(save_folder,run+1))
                best_model = copy.deepcopy(model)
                best_loss = avg_validation_loss
                not_improved_count = 0
            else:
                not_improved_count += 1
        # Early stopping
        if not_improved_count == early_stop_patience:
            print("Validation performance didn\'t improve for {} epochs. "
                            "Training stops.".format(early_stop_patience))
            break
    training_time = time.time() - start_time
    print("Training time:", training_time)
    filepath = '{}/Loss_'.format(save_folder)+data_name+'_run{}.txt'.format(run)    
    with open(filepath, 'w') as file:
        for item in zip(epochs_plt,mae_plt,valid_loss_plt):
            file.write("{}\n".format(item))
        file.write("Running time: {}\n".format(training_time))

## 5. Evaluation

In [ ]:
lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(tab_test)
prefixes, outputs = Extract_prefix(lines, lines_t, lines_t2, lines_t3, lines_t4)

In [ ]:
def evaluate_model(model):
    model.to(device)
    err_dict = {}
    with torch.no_grad():
        model.eval()
        testing_loss_all = 0
        num_of_minibatch = 0
        for i,(inputs,targets) in enumerate(test_loader):
            prefix_len = len(prefixes[0][i])
            targets = targets.to(device)
            yhat = model(inputs.to(device))
            loss_mape = np.abs((targets.item() - yhat.item()/targets.item()))*100
            criterion = nn.L1Loss()
            loss_mae = criterion(yhat,targets).item()
            if prefix_len not in err_dict.keys():
                err_dict[prefix_len] = [[loss_mape, loss_mae]]
            else:
                err_dict[prefix_len].append([loss_mape, loss_mae])
    return err_dict

In [ ]:
err_total_dict = {}
print(save_folder)
for run in range(5):
    print("Run: {}".format(run+1))
    trained_model = LSTM_direct_model(hidden_dim, num_layers, droppout_prob)
    trained_model.load_state_dict(torch.load('{}/best_model_run_{}.pt'.format(save_folder,run+1),
                                         map_location=torch.device(device)))
    err_dict = evaluate_model(trained_model)
    
    for key in err_dict.keys():
        err = torch.mean(torch.tensor(err_dict[key]), axis = 0)
        if key in err_total_dict.keys():
            err_total_dict[key].append(torch.tensor([err[0], err[1]*divisor_rt/86400]))
        else:
            err_total_dict[key] = [torch.tensor([err[0], err[1]*divisor_rt/86400])]

In [ ]:
num_samples_dict = {}
for i,(inputs,targets) in enumerate(test_loader):
    key = len(prefixes[0][i])
    if key in num_samples_dict.keys():
        num_samples_dict[key] += 1
    else:
        num_samples_dict[key] = 1

In [ ]:
list_prefix_len = []
list_num_samples = []
list_mape_err = []
list_mape_std = []
list_mae_err = []
list_mae_std = []
for key, value in err_total_dict.items():
    list_prefix_len.append(key)
    list_num_samples.append(num_samples_dict[key])
    list_mape_err.append(round(torch.stack(err_total_dict[key]).mean(axis = 0)[0].item(), 3))
    list_mape_std.append(round(torch.stack(err_total_dict[key]).std(axis=0)[0].item(), 3))
    list_mae_err.append(round(torch.stack(err_total_dict[key]).mean(axis = 0)[1].item(), 3))
    list_mae_std.append(round(torch.stack(err_total_dict[key]).std(axis=0)[1].item(), 3))
tab_result = pd.DataFrame({"Prefix length":list_prefix_len, "Num samples": list_num_samples, 
                           "MAPE(%)":list_mape_err, "MAPE std": list_mape_std,
                           "MAE(days)": list_mae_err, "MAE std": list_mae_std})
tab_result

In [ ]:
tab = tab_result[tab_result["Num samples"] >= 20]
sum(tab["Num samples"]*tab["MAE(days)"])/sum(tab["Num samples"])

In [ ]:
tab_result.to_csv(project_dir+"4_Outputs/Evaluation/"+data_name+"_Tax_LSTM_eval.csv", index = False)